<a href="https://colab.research.google.com/github/Lekhaaa14/AI_PLAGIARISM_CHECKER/blob/main/ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AI Content Detector using GLM-4.7-Flash via Hugging Face API
# Detects AI-generated content and highlights it in red

# Install required libraries
!pip install requests PyMuPDF -q

import requests
import json
import re
import fitz  # PyMuPDF
from google.colab import files
import io
from getpass import getpass
import time

class AIContentDetectorGLM:
    def __init__(self, hf_api_key):
        """Initialize AI Content Detector with Hugging Face API"""
        self.api_key = hf_api_key
        self.api_url = "https://api-inference.huggingface.co/models/zai-org/GLM-4.7-Flash"
        self.headers = {"Authorization": f"Bearer {hf_api_key}"}
        print("✅ AI Content Detector initialized with Hugging Face API")
        print(f"📦 Model: GLM-4.7-Flash (Zhipu AI)\n")

    def query_model(self, prompt, max_retries=3):
        """Query the GLM-4.7-Flash model via Hugging Face API"""
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 300,
                "temperature": 0.1,
                "top_p": 0.9,
                "return_full_text": False
            }
        }

        for attempt in range(max_retries):
            try:
                response = requests.post(self.api_url, headers=self.headers, json=payload)

                # Check if model is loading
                if response.status_code == 503:
                    result = response.json()
                    if 'estimated_time' in result:
                        wait_time = result['estimated_time']
                        print(f"⏳ Model loading... waiting {wait_time:.0f}s", end='\r')
                        time.sleep(wait_time + 2)
                        continue

                if response.status_code == 200:
                    result = response.json()

                    # Handle different response formats
                    if isinstance(result, list) and len(result) > 0:
                        if isinstance(result[0], dict) and 'generated_text' in result[0]:
                            return result[0]['generated_text']
                        elif isinstance(result[0], str):
                            return result[0]
                    elif isinstance(result, dict) and 'generated_text' in result:
                        return result['generated_text']
                    elif isinstance(result, str):
                        return result

                    return str(result)
                else:
                    print(f"\n⚠️ API Error {response.status_code}: {response.text}")

            except Exception as e:
                print(f"\n⚠️ Attempt {attempt + 1} failed: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2)

        return None

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF"""
        try:
            doc = fitz.open(pdf_path)
            text = ""

            for page_num in range(len(doc)):
                page = doc[page_num]
                text += page.get_text()

            doc.close()
            return text.strip()
        except Exception as e:
            print(f"❌ Error extracting PDF: {str(e)}")
            return None

    def split_into_sentences(self, text):
        """Split text into sentences"""
        sentences = re.split(r'[.!?]+', text)
        sentences = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 10]
        return sentences

    def detect_ai_sentence(self, sentence):
        """Detect if a single sentence is AI-generated using GLM-4.7-Flash"""
        prompt = f"""Analyze if this sentence is written by AI or human.

Sentence: "{sentence}"

Respond ONLY with JSON in this exact format:
{{"is_ai_generated": true or false, "confidence": 0-100, "reasoning": "brief explanation"}}

JSON:"""

        response_text = self.query_model(prompt)

        if not response_text:
            return {'is_ai_generated': False, 'confidence': 0, 'reasoning': 'Error querying model'}

        try:
            # Try to extract JSON from response
            json_match = re.search(r'\{[^}]*"is_ai_generated"[^}]*\}', response_text, re.DOTALL)
            if json_match:
                result = json.loads(json_match.group())
                return result

            # If no JSON found, try parsing the whole response
            result = json.loads(response_text)
            return result

        except Exception as e:
            # Fallback: parse response text for keywords
            response_lower = response_text.lower()

            is_ai = any(word in response_lower for word in ['ai-generated', 'ai generated', 'artificial', 'true'])
            confidence = 50

            # Try to extract confidence number
            conf_match = re.search(r'(\d{1,3})\s*%', response_text)
            if conf_match:
                confidence = int(conf_match.group(1))

            return {
                'is_ai_generated': is_ai,
                'confidence': confidence,
                'reasoning': response_text[:100]
            }

    def analyze_document(self, text):
        """Analyze entire document for AI-generated content"""
        print("\n🔍 Analyzing document with GLM-4.7-Flash...")
        print("="*70)

        sentences = self.split_into_sentences(text)
        total_sentences = len(sentences)

        print(f"📄 Total sentences to analyze: {total_sentences}")
        print("⏳ This may take a moment...\n")

        results = []
        ai_count = 0

        for i, sentence in enumerate(sentences, 1):
            print(f"Analyzing sentence {i}/{total_sentences}...", end='\r')

            result = self.detect_ai_sentence(sentence)

            sentence_result = {
                'sentence': sentence,
                'is_ai': result.get('is_ai_generated', False),
                'confidence': result.get('confidence', 0),
                'reasoning': result.get('reasoning', '')
            }

            results.append(sentence_result)

            if sentence_result['is_ai']:
                ai_count += 1

            # Small delay to avoid rate limiting
            time.sleep(0.5)

        print("\n")

        ai_percentage = (ai_count / total_sentences * 100) if total_sentences > 0 else 0

        return {
            'total_sentences': total_sentences,
            'ai_sentences': ai_count,
            'human_sentences': total_sentences - ai_count,
            'ai_percentage': round(ai_percentage, 2),
            'sentence_results': results
        }

    def highlight_pdf(self, input_pdf_path, analysis_results, output_filename="ai_detection_report.pdf"):
        """Highlight AI content in original PDF preserving exact format"""
        print("\n📝 Highlighting AI content in original PDF...")

        try:
            # Open the original PDF
            doc = fitz.open(input_pdf_path)

            # Build set of AI sentences
            ai_sentences = []
            for result in analysis_results['sentence_results']:
                if result['is_ai']:
                    ai_sentences.append(result['sentence'].strip())

            # Process each page
            for page_num in range(len(doc)):
                page = doc[page_num]

                # Search for each AI sentence and highlight it
                for ai_sentence in ai_sentences:
                    # Try to find the sentence or parts of it
                    words = ai_sentence.split()

                    # Try highlighting with different word combinations
                    for i in range(len(words)):
                        for j in range(i+3, min(i+15, len(words)+1)):  # 3-15 words
                            search_phrase = ' '.join(words[i:j])

                            # Search for this phrase
                            text_instances = page.search_for(search_phrase)

                            # Highlight all instances
                            for inst in text_instances:
                                highlight = page.add_highlight_annot(inst)
                                highlight.set_colors(stroke=(1, 0, 0))  # Red color
                                highlight.update()

            # Save the highlighted PDF
            doc.save(output_filename, garbage=4, deflate=True, clean=True)
            doc.close()

            print(f"✅ Highlighted PDF saved as: {output_filename}")

        except Exception as e:
            print(f"❌ Error highlighting PDF: {str(e)}")
            print("Creating summary report instead...")
            self.create_summary_pdf(analysis_results, output_filename)

    def create_summary_pdf(self, analysis_results, output_filename):
        """Create a summary PDF if highlighting fails"""
        from reportlab.lib.pagesizes import letter
        from reportlab.lib import colors
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
        from reportlab.lib.units import inch

        doc = SimpleDocTemplate(output_filename, pagesize=letter)
        story = []
        styles = getSampleStyleSheet()

        title_style = ParagraphStyle(
            'Title',
            parent=styles['Heading1'],
            fontSize=24,
            alignment=1
        )

        story.append(Paragraph("AI CONTENT DETECTION SUMMARY", title_style))
        story.append(Spacer(1, 0.5*inch))

        ai_pct = analysis_results['ai_percentage']
        summary = f"""
        <b>Model Used:</b> GLM-4.7-Flash (Zhipu AI)<br/>
        <br/>
        <b>Total Sentences:</b> {analysis_results['total_sentences']}<br/>
        <b>AI-Generated:</b> {analysis_results['ai_sentences']}<br/>
        <b>Human-Written:</b> {analysis_results['human_sentences']}<br/>
        <br/>
        <b><font size=16 color={'red' if ai_pct > 50 else 'green'}>
        Overall AI Detection: {ai_pct}%
        </font></b>
        """

        story.append(Paragraph(summary, styles['Normal']))
        story.append(Spacer(1, 0.3*inch))

        story.append(Paragraph("<b>AI-DETECTED SENTENCES:</b>", styles['Heading2']))
        story.append(Spacer(1, 0.2*inch))

        for i, result in enumerate(analysis_results['sentence_results'], 1):
            if result['is_ai']:
                text = f"<font color=red><b>[{result['confidence']}%]</b> {result['sentence']}</font>"
                story.append(Paragraph(text, styles['Normal']))
                story.append(Spacer(1, 0.1*inch))

        doc.build(story)
        print(f"✅ Summary PDF created: {output_filename}")

    def print_console_report(self, analysis_results):
        """Print analysis results to console"""
        print("\n" + "="*70)
        print("AI CONTENT DETECTION REPORT - GLM-4.7-Flash")
        print("="*70)

        print(f"\n📊 SUMMARY:")
        print(f"   Total Sentences: {analysis_results['total_sentences']}")
        print(f"   AI-Generated: {analysis_results['ai_sentences']}")
        print(f"   Human-Written: {analysis_results['human_sentences']}")

        ai_pct = analysis_results['ai_percentage']
        status = "⚠️ HIGH" if ai_pct > 50 else "✅ LOW"
        print(f"\n   Overall AI Detection: {ai_pct}% {status}")

        print(f"\n📝 AI-DETECTED SENTENCES:")
        print("-"*70)

        for i, result in enumerate(analysis_results['sentence_results'], 1):
            if result['is_ai']:
                print(f"\n🔴 [AI-{result['confidence']}%] Sentence {i}:")
                print(f"   {result['sentence']}")
                print(f"   Reason: {result['reasoning']}")

        print("\n" + "="*70)


# Main function for PDF upload
def run_pdf_analysis():
    """Main function to run AI content detection with PDF upload"""

    print("\n" + "="*70)
    print("🤖 AI CONTENT DETECTOR - GLM-4.7-Flash Model")
    print("="*70 + "\n")

    # Get API key securely
    print("🔑 Enter your Hugging Face API key:")
    HF_API_KEY = getpass("(input hidden for security) ")

    if not HF_API_KEY or len(HF_API_KEY) < 10:
        print("❌ Invalid API key!")
        return

    try:
        # Initialize detector
        detector = AIContentDetectorGLM(HF_API_KEY)

        # Upload PDF
        print("\n📤 Please upload your PDF file:")
        uploaded = files.upload()

        if not uploaded:
            print("❌ No file uploaded!")
            return

        # Get the first uploaded file
        filename = list(uploaded.keys())[0]
        pdf_content = uploaded[filename]

        print(f"\n✅ File uploaded: {filename}")

        # Save uploaded file temporarily
        temp_path = "/tmp/input.pdf"
        with open(temp_path, 'wb') as f:
            f.write(pdf_content)

        # Extract text from PDF
        print("📄 Extracting text from PDF...")
        text = detector.extract_text_from_pdf(temp_path)

        if not text:
            print("❌ Could not extract text from PDF!")
            return

        print(f"✅ Extracted {len(text)} characters")

        # Analyze document
        results = detector.analyze_document(text)

        # Print console report
        detector.print_console_report(results)

        # Highlight in original PDF
        output_path = "/tmp/ai_detection_report.pdf"
        detector.highlight_pdf(temp_path, results, output_path)

        # Download the report
        print("\n📥 Downloading report...")
        files.download(output_path)

        print("\n" + "="*70)
        print("✅ ANALYSIS COMPLETE!")
        print("="*70)
        print("\n💡 Check the downloaded PDF:")
        print("   • Original PDF format preserved")
        print("   • AI-generated content highlighted in RED")
        print("   • Powered by GLM-4.7-Flash model")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        print("Please check your API key and try again.")


# Test function
def run_test_analysis():
    """Test mode - Creates a sample PDF and analyzes it"""

    print("\n" + "="*70)
    print("🧪 AI CONTENT DETECTOR - TEST MODE (GLM-4.7-Flash)")
    print("="*70 + "\n")

    # Get API key
    print("🔑 Enter your Hugging Face API key:")
    HF_API_KEY = getpass("(input hidden for security) ")

    if not HF_API_KEY or len(HF_API_KEY) < 10:
        print("❌ Invalid API key!")
        return

    try:
        from reportlab.pdfgen import canvas
        from reportlab.lib.pagesizes import letter

        detector = AIContentDetectorGLM(HF_API_KEY)

        # Create a sample PDF
        test_pdf_path = "/tmp/test_input.pdf"
        c = canvas.Canvas(test_pdf_path, pagesize=letter)

        sample_texts = [
            "Artificial intelligence has revolutionized numerous industries.",
            "I personally think AI is kinda cool but scary, ya know?",
            "The implementation of machine learning demonstrates capabilities.",
            "My dog ate my homework last Tuesday and I was mad!"
        ]

        y_position = 750
        c.setFont("Helvetica", 12)

        for text in sample_texts:
            c.drawString(50, y_position, text)
            y_position -= 30

        c.save()

        print("📝 Created sample PDF for testing...\n")

        # Extract and analyze
        text = detector.extract_text_from_pdf(test_pdf_path)
        results = detector.analyze_document(text)

        # Print results
        detector.print_console_report(results)

        # Highlight PDF
        output_path = "/tmp/test_ai_detection_report.pdf"
        detector.highlight_pdf(test_pdf_path, results, output_path)

        # Download report
        print("\n📥 Downloading report...")
        files.download(output_path)

        print("\n✅ Test complete! Check the downloaded PDF.")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()


# Simple menu to choose mode
def main():
    """Main menu"""
    print("\n" + "="*70)
    print("🤖 AI CONTENT DETECTOR - GLM-4.7-Flash (Hugging Face)")
    print("="*70)
    print("\nChoose an option:")
    print("1. Upload PDF and analyze")
    print("2. Test with sample PDF")

    choice = input("\nEnter your choice (1 or 2): ").strip()

    if choice == "1":
        run_pdf_analysis()
    elif choice == "2":
        run_test_analysis()
    else:
        print("❌ Invalid choice! Please run again and choose 1 or 2.")


# Run the program
print("\n🚀 Starting AI Content Detector with GLM-4.7-Flash...")
print("="*70)
main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 36.4 MB/s eta 0:00:00

🚀 Starting AI Content Detector with GLM-4.7-Flash...

🤖 AI CONTENT DETECTOR - GLM-4.7-Flash (Hugging Face)

Choose an option:
1. Upload PDF and analyze
2. Test with sample PDF

Enter your choice (1 or 2): 1

🤖 AI CONTENT DETECTOR - GLM-4.7-Flash Model

🔑 Enter your Hugging Face API key:
(input hidden for security) ··········
✅ AI Content Detector initialized with Hugging Face API
📦 Model: GLM-4.7-Flash (Zhipu AI)


📤 Please upload your PDF file:


Saving multimodel.pdf to multimodel.pdf

✅ File uploaded: multimodel.pdf
📄 Extracting text from PDF...
✅ Extracted 22340 characters

🔍 Analyzing document with GLM-4.7-Flash...
📄 Total sentences to analyze: 207
⏳ This may take a moment...

Analyzing sentence 1/207...
⚠️ API Error 410: {"error":"https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead."}

⚠️ API Error 410: {"error":"https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead."}

⚠️ API Error 410: {"error":"https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead."}
Analyzing sentence 2/207...
⚠️ API Error 410: {"error":"https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead."}

⚠️ API Error 410: {"error":"https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ ANALYSIS COMPLETE!

💡 Check the downloaded PDF:
   • Original PDF format preserved
   • AI-generated content highlighted in RED
   • Powered by GLM-4.7-Flash model
